In [1]:
import random,subprocess
def generate_model_line(s,idx): #generates svm input vectors using predefined character numbers
    sentence=[x for x in list(''.join(set(''.join(s.split('\t')[0])))) if x not in insignificant_tokens]
    model_line =str(idx+1)
    character_list=[]
    for ch in sentence:
        character_list.append(character_numbers[ch])
    character_list.sort()
    for ch in character_list:
        model_line+=" "+ str(ch)+ ":" + str(1)
    return model_line+"\n"

insignificant_tokens=[' ','!', '"','#','$','%','&','*','+','-','(',')',',','.','/','0','1','2','3','4','5','6','7','8',
                      '9',';','<','>','=','?','@','|','«','»','`','[',']',"'",'\\']#these tokens will be skipped 
language_ids=['bg','bs','cz','es-AR','es-ES','hr','id','mk','my','pt-BR','pt-PT','sk','sr']
with open ("Corpus/Raw Corpus.txt") as f:
    corpus = f.readlines()
languages=[]
corpus_s=list(''.join(set(''.join(corpus))))
corpus_s=[x for x in corpus_s if x not in insignificant_tokens]
character_numbers={}
for s in corpus_s:
    character_numbers[s]=ord(s) #assign each character to a number using ordinal of a one-character string.

for i in range(13): #divides data set as languages
    languages.append("")
    start=i*2000
    end=(i+1)*2000
    languages[i]=corpus[start:end] #2k sentence for each language

training_set=[]
test_set=[]
training_model=[]
test_model=[]
for idx,l in enumerate(languages): #assign training and test partitions to each language
    random.shuffle(l)
    
    training_partition=l[0:1800]
    training_set.extend(training_partition)
    
    for s in training_partition:
        training_model.append(generate_model_line(s,idx))
    
    test_partition=l[1800:2000]
    test_set.extend(test_partition)
    
    for s in test_partition:
        test_model.append(generate_model_line(s,idx))

In [2]:
def write_files(file_name,array):
    with open (file_name, mode='wt') as t_file:
        for item in array:
            t_file.write(item)
print(len(training_set))
print(len(test_set))
write_files("SVM/TrainingModel-SVM.txt",training_model) #svm inputs as training set
write_files("SVM/TrainingSet-SVM.txt",training_set) #which sentences in the training set
write_files("SVM/TestSet-SVM.txt",test_set) #svm inputs as test set
write_files("SVM/TestModel-SVM.txt",test_model) #which sentences in the test set

23400
2600


In [3]:
p=subprocess.Popen(['SVM/svm_multiclass_learn', '-c', '5000' ,'SVM/TrainingModel-SVM.txt' ,'SVM/Model'],
                 stdout=subprocess.PIPE) #train svm using shell command
p.wait()



0

In [4]:
p=subprocess.Popen(['SVM/svm_multiclass_classify', 'SVM/TestModel-SVM.txt' ,'SVM/Model','SVM/predictions.txt'],
                 stdout=subprocess.PIPE) #classify using test set using shell command
for line in p.stdout:
    print(line)
p.wait()


Reading model...done.

Reading test examples... (2600 examples) done.

Classifying test examples...done

Runtime (without IO) in cpu-seconds: 0.01

Average loss on test set: 31.6538

Zero/one-error on test set: 31.65% (1777 correct, 823 incorrect, 2600 total)



0

In [5]:
with open ("SVM/predictions.txt") as f:
    predictions = f.readlines()
predictions=[i.split(' ')[0] for i in predictions]
metrics={}
for idx,lang_id in enumerate(language_ids):
    false_negatives=len([x for x in predictions[idx*200:(idx+1)*200] if int(x) != (idx+1)])
    true_positives=200-false_negatives
    if idx==0: #count false positives on proceeding predictions
        false_positives=len([x for x in predictions[200:len(predictions)] if int(x) ==1])
    elif idx==12: #count false positives on preceeding predictions
        false_positives=len([x for x in predictions[0:2400] if int(x)==13])
    else: #count false positives on both preceeding and proceeding predictions
        false_positives=[x for x in predictions[(idx-1)*200:idx*200] if int(x) == (idx+1)]
        false_positives=len(false_positives + [x for x in predictions[(idx+1)*200:2600] if int(x) == (idx+1)])
    true_negatives=2400-false_positives
    metrics[lang_id]={"tp":true_positives,"fn":false_negatives,"tn":true_negatives,"fp":false_positives}
for key,value in metrics.iteritems():
    print(key)
    print("True positives: " + str(value["tp"]))
    print("False positives: " + str(value["fp"]))
    print("True negatives: " + str(value["tn"]))
    print("False negatives: " + str(value["fn"]))
    

bg
True positives: 196
False positives: 0
True negatives: 2400
False negatives: 4
hr
True positives: 124
False positives: 74
True negatives: 2326
False negatives: 76
es-AR
True positives: 71
False positives: 27
True negatives: 2373
False negatives: 129
mk
True positives: 200
False positives: 0
True negatives: 2400
False negatives: 0
sk
True positives: 195
False positives: 5
True negatives: 2395
False negatives: 5
cz
True positives: 195
False positives: 1
True negatives: 2399
False negatives: 5
sr
True positives: 98
False positives: 110
True negatives: 2290
False negatives: 102
pt-BR
True positives: 95
False positives: 63
True negatives: 2337
False negatives: 105
bs
True positives: 44
False positives: 42
True negatives: 2358
False negatives: 156
my
True positives: 143
False positives: 5
True negatives: 2395
False negatives: 57
pt-PT
True positives: 124
False positives: 92
True negatives: 2308
False negatives: 76
es-ES
True positives: 183
False positives: 129
True negatives: 2271
False n

In [6]:
total_tp=0.0
total_fp=0.0
total_fn=0.0
total_tn=0.0
total_precision=0.0
total_recall=0.0
total_f1score=0.0
for key in metrics.keys():
    tp=metrics[key]["tp"]
    fp=metrics[key]["fp"]
    fn=metrics[key]["fn"]
    tn=metrics[key]["tn"]
    precision=tp/float(tp+fp)
    recall=tp/float(tp+fn)
    f1_score=(2*recall*precision)/float(recall+precision)
    total_precision+=precision
    total_recall+=recall
    total_f1score+=f1_score
    
    total_tp+=tp
    total_fp+=fp
    total_fn+=fn
    total_tn+=tn
    
    #calc macro
#calc micro
mic_prec=total_tp/float(total_tp+total_fp)
mic_recall=total_tp/float(total_tp+total_fn)
print("Micro-averaged precision: " + str(mic_prec))
print("Micro-averaged recall: " + str(mic_recall))
print("Micro-averaged f1-score: " + str((2*mic_prec*mic_recall)/float(mic_prec+mic_recall)))
print("")
print("Macro-averaged precision: " + str(total_precision/13.0))
print("Macro-averaged recall: " + str(total_recall/13.0))
print("Macro-averaged f1-score: " + str(total_f1score/13.0))
print("")
print("Total accuracy: "+ str(total_tp/2600.0))
print("Accuracies for languages:")
for key,value in metrics.iteritems():
    print(key + str(": ")+str(value["tp"]/200.0))
print("")

print("fp: "+str(total_fp))
print("tp: "+str(total_tp))
print("fn: "+str(total_fn))
print("tn: "+str(total_tn))

metrics

Micro-averaged precision: 0.744449099288
Micro-averaged recall: 0.683461538462
Micro-averaged f1-score: 0.712652897534

Macro-averaged precision: 0.743765662284
Macro-averaged recall: 0.683461538462
Macro-averaged f1-score: 0.699124738159

Total accuracy: 0.683461538462
Accuracies for languages:
bg: 0.98
hr: 0.62
es-AR: 0.355
mk: 1.0
sk: 0.975
cz: 0.975
sr: 0.49
pt-BR: 0.475
bs: 0.22
my: 0.715
pt-PT: 0.62
es-ES: 0.915
id: 0.545

fp: 610.0
tp: 1777.0
fn: 823.0
tn: 30590.0


{'bg': {'fn': 4, 'fp': 0, 'tn': 2400, 'tp': 196},
 'bs': {'fn': 156, 'fp': 42, 'tn': 2358, 'tp': 44},
 'cz': {'fn': 5, 'fp': 1, 'tn': 2399, 'tp': 195},
 'es-AR': {'fn': 129, 'fp': 27, 'tn': 2373, 'tp': 71},
 'es-ES': {'fn': 17, 'fp': 129, 'tn': 2271, 'tp': 183},
 'hr': {'fn': 76, 'fp': 74, 'tn': 2326, 'tp': 124},
 'id': {'fn': 91, 'fp': 62, 'tn': 2338, 'tp': 109},
 'mk': {'fn': 0, 'fp': 0, 'tn': 2400, 'tp': 200},
 'my': {'fn': 57, 'fp': 5, 'tn': 2395, 'tp': 143},
 'pt-BR': {'fn': 105, 'fp': 63, 'tn': 2337, 'tp': 95},
 'pt-PT': {'fn': 76, 'fp': 92, 'tn': 2308, 'tp': 124},
 'sk': {'fn': 5, 'fp': 5, 'tn': 2395, 'tp': 195},
 'sr': {'fn': 102, 'fp': 110, 'tn': 2290, 'tp': 98}}

In [7]:
for key,value in metrics.iteritems():
    print(key + "\t"+ str(value["tp"]) + "\t"+ str(value["fp"])+"\t"+str(value["tn"])+"\t"+str(value["fn"]))
    

bg	196	0	2400	4
hr	124	74	2326	76
es-AR	71	27	2373	129
mk	200	0	2400	0
sk	195	5	2395	5
cz	195	1	2399	5
sr	98	110	2290	102
pt-BR	95	63	2337	105
bs	44	42	2358	156
my	143	5	2395	57
pt-PT	124	92	2308	76
es-ES	183	129	2271	17
id	109	62	2338	91


In [9]:
print("Accuracies for languages:")
for key,value in metrics.iteritems():
    print(str(value["tp"]/200.0))
print("")

Accuracies for languages:
0.98
0.62
0.355
1.0
0.975
0.975
0.49
0.475
0.22
0.715
0.62
0.915
0.545

